# Número de Casos por CC.AA. hasta el día actual

In [ ]:
import pandas as pd
import io
import requests
from datetime import date

url_general = 'https://covid19.isciii.es/resources/data.csv'
url_ccaa = 'https://covid19.isciii.es/resources/ccaa.csv'

s = requests.get(url_general, verify=False).content
general = pd.read_csv(io.StringIO(s.decode('utf-8')))

t = requests.get(url_ccaa, verify=False).content
cc_aa = pd.read_csv(io.StringIO(t.decode('utf-8')))

In [347]:
general

,Fecha,Hora,Casos,Defunciones,Recuperados
0,17 de marzo de 2020,20:00,13716,598,1081


In [348]:
cc_aa

,CCAA,Nº Casos
0,Andalucía,859
1,Aragón,226
2,Asturias,242
3,Baleares,112
4,Canarias,181
5,Cantabria,68
6,Castilla La Mancha,675
7,Castilla y León,668
8,Cataluña,1866
9,Ceuta,1


In [349]:
hoy = date.today().strftime('%d/%m/%Y')

In [350]:
import csv
general.to_csv(date.today().strftime('%Y%m%d') + '_general.csv')
cc_aa.to_csv(date.today().strftime('%Y%m%d') + '_ccaa.csv')

In [351]:
cc_aa=cc_aa.sort_values(by='Nº Casos', ascending=True)
ccaa = cc_aa['CCAA']
casos = cc_aa['Nº Casos']

In [352]:
casos

9        1
14      20
5       68
3      112
15     122
4      181
11     194
1      226
2      242
12     341
16     386
18     419
7      668
6      675
10     726
0      859
17     973
8     1866
13    5637
Name: Nº Casos, dtype: int64

In [353]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x=casos,
    y=ccaa,
    text=casos,
    textposition='outside',
    orientation='h',
    name="casos_ccaa"
))

fig.update_layout(
    title="Número de casos por Comunidades Autónomas hasta el " + hoy,
    xaxis_title="Número casos",
    yaxis_title="Comunidades Autónomas",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#000000"
    )
)

fig.show()

# Porcenatje variación casos por CC.AA desde el día anterior al actual

In [321]:
from datetime import datetime
from datetime import timedelta
import json

now = datetime.now()
new_date = now + timedelta(days=-1)
ayer = new_date.strftime('%Y%m%d')
ayer

'20200317'

In [322]:
 casos_dia_anterior = pd.read_csv('20200317_ccaa.csv', index_col=None)

In [323]:
casos_dia_anterior

,Unnamed: 0,CCAA,Nº Casos
0,9,Ceuta,1
1,14,Melilla,10
2,5,Cantabria,60
3,3,Baleares,102
4,15,Murcia,121
5,4,Canarias,100
6,11,Extremadura,194
7,1,Aragón,200
8,2,Asturias,202
9,12,Galicia,321


In [324]:
variacion_porcentaje = {}
variacion_absoluta = {}
for i in range(19):
    ccaa = cc_aa['CCAA'][i]
    casos = cc_aa['Nº Casos'][i]
    for j in range(19):
        ccaa_ayer = casos_dia_anterior['CCAA'][j]
        casos_ayer = int(casos_dia_anterior['Nº Casos'][j])
        if ccaa_ayer == ccaa:
            variacion = (int(casos) - int(casos_ayer))/int(casos_ayer) * 100
            variacion_porcentaje[cc_aa['CCAA'][i]] = str(round(variacion, 2))
            variacion_absoluta[cc_aa['CCAA'][i]] = variacion
variacion_porcentaje

{'Andalucía': '30.35',
 'Aragón': '13.0',
 'Asturias': '19.8',
 'Baleares': '9.8',
 'Canarias': '81.0',
 'Cantabria': '13.33',
 'Castilla La Mancha': '4.65',
 'Castilla y León': '4.7',
 'Cataluña': '60.03',
 'Ceuta': '0.0',
 'C. Valenciana': '-14.59',
 'Extremadura': '0.0',
 'Galicia': '6.23',
 'Madrid': '40.92',
 'Melilla': '100.0',
 'Murcia': '0.83',
 'Navarra': '0.0',
 'País Vasco': '5.42',
 'La Rioja': '2.44'}

In [325]:
tabla_variacion = pd.DataFrame([[key, variacion_porcentaje[key]] for key in variacion_porcentaje.keys()], columns=['CCAA', '% Variacion'])

In [326]:
tabla_variacion

,CCAA,% Variacion
0,Andalucía,30.35
1,Aragón,13.0
2,Asturias,19.8
3,Baleares,9.8
4,Canarias,81.0
5,Cantabria,13.33
6,Castilla La Mancha,4.65
7,Castilla y León,4.7
8,Cataluña,60.03
9,Ceuta,0.0


In [327]:
comunidades = tabla_variacion['CCAA']
var = tabla_variacion['% Variacion']

clrrojo = '#ff0000'
clrverde = '#00ff08'
clrs  = [clrverde if float(var[x]) < 0 else clrrojo for x in range(len(var))]

fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=comunidades,
    y=var,
    textposition='outside',
    text=var + '%',
    marker=dict(color=clrs),
    name="variacion_ccaa"
))

fig2.update_layout(
    title="Variacion (%) del número de casos por CC.AA. respecto a ayer",
    xaxis_title="Comunidades Autónomas",
    yaxis_title="% Variación",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#000000"
    )
)

fig2.show()

In [337]:
s = requests.get('https://as.com/').content

OSError: Could not find a suitable TLS CA certificate bundle, invalid path: /home/erebus/anaconda3/lib/python3.7/site-packages/certifi/cacert.pem